# ES to BBF BAN Creation (v9)

Creates new BBF billing accounts for ES BANs that have orders moving to BBF.

## v9 Changes from v8
- **Idempotency:** Checks if -BBF BAN already exists (via `legacy_es_id__c`) before creating
- **Fix missing customer links:** Option to update OSS `customers.customers.salesforce_id` when missing
- Can be run weekly without creating duplicates

## Process Overview
1. Query all `billing_invoice__c` with active orders
2. Check if -BBF BAN already exists for each (skip if yes)
3. Classify orders as WD or BBF
4. Optionally fix missing OSS customer salesforce_id links
5. Create -BBF accounts for BANs that need them

## Classification Logic
| BAN Type | Action |
|----------|--------|
| WD-only (all PA MARKET DECOM) | SKIP - no new account |
| BBF-only | Create 1 -BBF account |
| Mixed | Create 1 -BBF account |
| Already has -BBF BAN | SKIP - already created |

## Safety
- `DRY_RUN = True` by default
- `FIX_MISSING_CUSTOMER_LINKS = False` by default

In [15]:
# === SETUP ===
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
from collections import defaultdict
import time
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Entity mapping
entity_map = {
    "Everstream Ohio": "EVO",
    "Everstream Michigan": "EVM",
    "OneCommunity": "EVO",
    "Medina County Fiber Network Services": "OCO",
    "Everstream Lynx": "EVL",
    "Michigan-LB": "EVB",
    "Michigan-RF": "EVR",
    "Everstream PA": "EVP",
}

# PA Markets
PA_MARKETS = ["Pittsburgh", "Harrisburg", "Philadelphia", "Scranton", "Uniti-PA"]

# WD Classification
WD_PROJECT_GROUP = "PA MARKET DECOM"

# Active statuses
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# Connection credentials
heroku_conn_kwargs = {
    "dbname": "d88otjf7uhv9pr",
    "user": "ucn7cbk14sd6h",
    "password": "pf27d102f95e996e621e02523d035a1bff27590c8e6a13f5b180703a6631320c5",
    "host": "ec2-54-86-217-174.compute-1.amazonaws.com",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

oss_conn_kwargs = {
    "dbname": "GLC",
    "user": "oss_server",
    "password": "3wU3uB28X?!r2?@ebrUg",
    "host": "pg01.comlink.net",
    "port": "5432",
    "cursor_factory": RealDictCursor,
    "connect_timeout": 10,
}

print("Connecting to Heroku...")
conn = psycopg2.connect(**heroku_conn_kwargs)
print("✅ Connected to Heroku")

print("Connecting to OSS...")
oconn = psycopg2.connect(**oss_conn_kwargs)
print("✅ Connected to OSS")

Connecting to Heroku...
✅ Connected to Heroku
Connecting to OSS...
✅ Connected to OSS


In [16]:
# === CONFIGURATION ===
DRY_RUN = False  # Set to False to actually create accounts
FIX_MISSING_CUSTOMER_LINKS = True  # Set to True to update missing salesforce_id in OSS
OUTPUT_DIR = "."

# Output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_type = "dry_run" if DRY_RUN else "live_run"
output_file = f"{OUTPUT_DIR}/es_to_bbf_ban_creation_v9_{run_type}_{timestamp}.xlsx"

print(f"DRY_RUN: {DRY_RUN}")
print(f"FIX_MISSING_CUSTOMER_LINKS: {FIX_MISSING_CUSTOMER_LINKS}")
print(f"Output file: {output_file}")

if not DRY_RUN:
    print("\n⚠️  WARNING: LIVE RUN - Accounts will be created!")

DRY_RUN: False
FIX_MISSING_CUSTOMER_LINKS: True
Output file: ./es_to_bbf_ban_creation_v9_live_run_20260106_093058.xlsx

⚠️  WARNING: LIVE RUN - Accounts will be created!


In [17]:
# === STEP 1: Check for existing -BBF BANs (idempotency) ===

existing_bbf_bans_sql = """
SELECT 
    legacy_es_id__c AS old_ban_sfid,
    sfid AS existing_bbf_sfid,
    name AS existing_bbf_name,
    account_number__c AS existing_account_id
FROM sfprod.billing_invoice__c
WHERE legacy_es_id__c IS NOT NULL
  AND bbf_ban__c = true
"""

print("Checking for existing -BBF BANs...")
existing_bbf_df = pd.read_sql_query(existing_bbf_bans_sql, conn)
existing_bbf_old_sfids = set(existing_bbf_df["old_ban_sfid"].tolist())

print(f"✅ Found {len(existing_bbf_df)} existing -BBF BANs (will skip these)")

Checking for existing -BBF BANs...
✅ Found 0 existing -BBF BANs (will skip these)


C:\Users\vjero\AppData\Local\Temp\ipykernel_37640\2764418319.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  existing_bbf_df = pd.read_sql_query(existing_bbf_bans_sql, conn)


In [18]:
# === STEP 2: Get all billing_invoice__c with active services and order classification ===

pa_markets_str = "','".join(PA_MARKETS)
status_str = "','".join(ACTIVE_STATUSES)

active_billing_accounts_sql = f"""
SELECT 
    bi.account_number__c AS old_account_id,
    bi.sfid AS old_billing_invoice_sfid,
    bi.account__c AS customer_sfid,
    bi.name AS old_ban_name,
    bi.account_name__c,
    bi.billing_address_1__c,
    bi.billing_city__c,
    bi.billing_state__c,
    bi.billing_zip__c,
    bi.billing_e_mail__c,
    bi.late_fee_percentage__c,
    bi.disable_late_fees__c,
    bi.payment_terms__c,
    bi.invoice_delivery_preference__c,
    COUNT(DISTINCT o.sfid) AS total_active_orders,
    -- WD orders: PA market AND Project_Group__c contains 'PA MARKET DECOM'
    COUNT(DISTINCT CASE 
        WHEN o.dimension_4_market__c IN ('{pa_markets_str}') 
             AND o.project_group__c LIKE '%{WD_PROJECT_GROUP}%'
        THEN o.sfid 
    END) AS wd_order_count,
    -- BBF orders: Everything else
    COUNT(DISTINCT CASE 
        WHEN NOT (
            o.dimension_4_market__c IN ('{pa_markets_str}') 
            AND o.project_group__c LIKE '%{WD_PROJECT_GROUP}%'
        )
        THEN o.sfid 
    END) AS bbf_order_count
FROM sfprod.billing_invoice__c bi
JOIN sfprod."order" o ON o.billing_invoice__c = bi.sfid
WHERE 
    o.status IN ('{status_str}')
    AND bi.disabled__c = false
    AND bi.account_number__c IS NOT NULL
GROUP BY 
    bi.account_number__c, bi.sfid, bi.account__c, bi.name, bi.account_name__c,
    bi.billing_address_1__c, bi.billing_city__c, bi.billing_state__c, 
    bi.billing_zip__c, bi.billing_e_mail__c, bi.late_fee_percentage__c, 
    bi.disable_late_fees__c, bi.payment_terms__c, bi.invoice_delivery_preference__c
ORDER BY bi.account_number__c;
"""

print("Executing query (this may take a moment)...")
with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(active_billing_accounts_sql)
    active_billing_accounts = cur.fetchall()

print(
    f"\nFound {len(active_billing_accounts)} ES billing accounts with active services"
)

Executing query (this may take a moment)...

Found 2520 ES billing accounts with active services


In [19]:
# === STEP 3: Filter out BANs that already have -BBF counterpart ===

already_has_bbf = []
needs_processing = []

for ba in active_billing_accounts:
    if ba["old_billing_invoice_sfid"] in existing_bbf_old_sfids:
        already_has_bbf.append(ba)
    else:
        needs_processing.append(ba)

print(f"\n=== Idempotency Check ===")
print(f"  Already has -BBF BAN (skip): {len(already_has_bbf)}")
print(f"  Needs processing: {len(needs_processing)}")


=== Idempotency Check ===
  Already has -BBF BAN (skip): 0
  Needs processing: 2520


In [20]:
# === STEP 4: Categorize BANs that need processing ===

wd_only = [
    ba
    for ba in needs_processing
    if ba["wd_order_count"] > 0 and ba["bbf_order_count"] == 0
]
bbf_only = [
    ba
    for ba in needs_processing
    if ba["bbf_order_count"] > 0 and ba["wd_order_count"] == 0
]
mixed = [
    ba
    for ba in needs_processing
    if ba["wd_order_count"] > 0 and ba["bbf_order_count"] > 0
]

# Only create accounts for BANs with BBF orders
bans_needing_bbf_account = bbf_only + mixed

print(f"\n=== BAN Classification (excluding already processed) ===")
print(f"  WD-only (PA MARKET DECOM): {len(wd_only)} → SKIP (no new account needed)")
print(f"  BBF-only: {len(bbf_only)} → Create -BBF account")
print(f"  Mixed: {len(mixed)} → Create -BBF account")
print(f"\n✅ Total -BBF accounts to create: {len(bans_needing_bbf_account)}")
print(f"⏭️  Skipping {len(wd_only)} WD-only BANs")
print(f"⏭️  Skipping {len(already_has_bbf)} BANs that already have -BBF")


=== BAN Classification (excluding already processed) ===
  WD-only (PA MARKET DECOM): 17 → SKIP (no new account needed)
  BBF-only: 2488 → Create -BBF account
  Mixed: 13 → Create -BBF account

✅ Total -BBF accounts to create: 2501
⏭️  Skipping 17 WD-only BANs
⏭️  Skipping 0 BANs that already have -BBF


In [21]:
# === STEP 5: Get SF Account data ===

customer_sfids = list(
    set([ba["customer_sfid"] for ba in bans_needing_bbf_account if ba["customer_sfid"]])
)
customer_sfids_str = "','".join(customer_sfids)

sf_accounts_sql = f"""
SELECT 
    sfid,
    name,
    billingstreet,
    billingcity,
    billingstate,
    billingpostalcode,
    email_address__c,
    onecommunity_entity__c,
    business_sector__c,
    send_consolidated_invoice__c
FROM sfprod.account
WHERE sfid IN ('{customer_sfids_str}')
"""

with conn.cursor(cursor_factory=RealDictCursor) as cur:
    cur.execute(sf_accounts_sql)
    sf_accounts = {row["sfid"]: row for row in cur.fetchall()}

print(f"Retrieved {len(sf_accounts)} SF Account records.")

Retrieved 2254 SF Account records.


In [22]:
# === STEP 6: Get OSS Customer records (by salesforce_id) ===

oss_customers_sql = f"""
SELECT 
    customer_id,
    customer_nm,
    salesforce_id
FROM customers.customers
WHERE salesforce_id IN ('{customer_sfids_str}')
  AND disabled >= now()
"""

with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
    ocur.execute(oss_customers_sql)
    oss_customers = {row["salesforce_id"]: row for row in ocur.fetchall()}

print(f"Retrieved {len(oss_customers)} OSS Customer records (by salesforce_id).")

# Find missing customers
missing_customer_sfids = [sfid for sfid in customer_sfids if sfid not in oss_customers]
if missing_customer_sfids:
    print(
        f"⚠️ Warning: {len(missing_customer_sfids)} SF Accounts have no OSS Customer record (by salesforce_id)"
    )
else:
    print("✅ All SF Accounts have matching OSS Customer records!")

Retrieved 2253 OSS Customer records (by salesforce_id).
⚠️ Warning: 1 SF Accounts have no OSS Customer record (by salesforce_id)


In [23]:
# === STEP 7: Try to find missing customers via account_id chain ===

customer_link_fixes = []  # Track fixes to make

if missing_customer_sfids:
    print(f"\n=== Attempting to find missing customers via account_id chain ===")

    # Get the account_number__c (old_account_id) for BANs with missing customers
    bans_with_missing_customers = [
        ba
        for ba in bans_needing_bbf_account
        if ba["customer_sfid"] in missing_customer_sfids
    ]

    print(f"  BANs affected: {len(bans_with_missing_customers)}")

    for ba in bans_with_missing_customers:
        old_account_id = ba["old_account_id"]
        customer_sfid = ba["customer_sfid"]

        # Look up customers.accounts by account_id to get customer_id
        with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
            ocur.execute(
                """
                SELECT a.account_id, a.customer_id, c.customer_nm, c.salesforce_id
                FROM customers.accounts a
                JOIN customers.customers c ON a.customer_id = c.customer_id
                WHERE a.account_id = %s
                  AND c.disabled >= now()
            """,
                (old_account_id,),
            )
            result = ocur.fetchone()

        if result:
            print(f"\n  Found via account_id chain:")
            print(f"    BAN account_number__c: {old_account_id}")
            print(f"    OSS customer_id: {result['customer_id']}")
            print(f"    OSS customer_nm: {result['customer_nm']}")
            print(f"    Current salesforce_id: {result['salesforce_id']}")
            print(f"    SF Account SFID to set: {customer_sfid}")

            if result["salesforce_id"] is None or result["salesforce_id"] == "":
                customer_link_fixes.append(
                    {
                        "customer_id": result["customer_id"],
                        "customer_nm": result["customer_nm"],
                        "old_account_id": old_account_id,
                        "sf_account_sfid": customer_sfid,
                        "current_salesforce_id": result["salesforce_id"],
                        "status": "NEEDS_FIX",
                    }
                )

                # Add to oss_customers so BAN creation can proceed
                oss_customers[customer_sfid] = {
                    "customer_id": result["customer_id"],
                    "customer_nm": result["customer_nm"],
                    "salesforce_id": customer_sfid,  # Will be set if FIX_MISSING_CUSTOMER_LINKS is True
                }
            else:
                print(
                    f"    ⚠️ salesforce_id already set to different value: {result['salesforce_id']}"
                )
        else:
            print(
                f"\n  ❌ Could not find OSS customer for account_id: {old_account_id}"
            )

    print(f"\n=== Customer Link Fixes Needed: {len(customer_link_fixes)} ===")


=== Attempting to find missing customers via account_id chain ===
  BANs affected: 1

  Found via account_id chain:
    BAN account_number__c: 117284
    OSS customer_id: 117285
    OSS customer_nm: MD9 Noblesville
    Current salesforce_id: None
    SF Account SFID to set: 001Qp00000r1Tz9IAE

=== Customer Link Fixes Needed: 1 ===


In [24]:
# === STEP 8: Apply customer link fixes (if enabled) ===

if customer_link_fixes:
    print(f"\n=== Customer Link Fixes ===")
    print(f"FIX_MISSING_CUSTOMER_LINKS: {FIX_MISSING_CUSTOMER_LINKS}")

    for fix in customer_link_fixes:
        if DRY_RUN or not FIX_MISSING_CUSTOMER_LINKS:
            fix["status"] = "WOULD_FIX" if FIX_MISSING_CUSTOMER_LINKS else "NEEDS_FIX"
            print(
                f"  [{fix['status']}] customer_id {fix['customer_id']} ({fix['customer_nm']}) → salesforce_id = {fix['sf_account_sfid']}"
            )
        else:
            # Actually apply the fix
            try:
                with oconn.cursor() as ocur:
                    ocur.execute(
                        """
                        UPDATE customers.customers
                        SET salesforce_id = %s
                        WHERE customer_id = %s
                    """,
                        (fix["sf_account_sfid"], fix["customer_id"]),
                    )
                oconn.commit()
                fix["status"] = "FIXED"
                print(
                    f"  [FIXED] customer_id {fix['customer_id']} ({fix['customer_nm']}) → salesforce_id = {fix['sf_account_sfid']}"
                )
            except Exception as e:
                oconn.rollback()
                fix["status"] = "ERROR"
                fix["error"] = str(e)
                print(f"  [ERROR] customer_id {fix['customer_id']}: {e}")
else:
    print("\nNo customer link fixes needed.")


=== Customer Link Fixes ===
FIX_MISSING_CUSTOMER_LINKS: True
  [FIXED] customer_id 117285 (MD9 Noblesville) → salesforce_id = 001Qp00000r1Tz9IAE


In [25]:
# === HELPER FUNCTIONS ===


def create_oss_account(sf_account_info, customer_id, ocur):
    """
    Creates a new OSS billing account using SF Account data.
    """
    # Parse zip code
    billing_zip = sf_account_info.get("billingpostalcode") or ""
    if "-" in billing_zip:
        zip_cd, zip4 = billing_zip.split("-", 1)
    else:
        zip_cd = billing_zip
        zip4 = None

    # Get entity code
    entity = sf_account_info.get("onecommunity_entity__c")
    if entity not in entity_map:
        return None, f"Unknown entity: {entity}"
    company_cd = entity_map[entity]

    # Determine customer type
    business_sector = sf_account_info.get("business_sector__c") or ""
    customer_type_cd = "W" if business_sector == "Wholesale" else "R"

    # Insert new account - bbf_ban is always True
    ocur.execute(
        """
        INSERT INTO customers.accounts (
            account_nm,
            customer_id,
            company_cd,
            address1,
            city,
            state_cd,
            zip,
            zip4,
            billing_email,
            created_by_id,
            late_fee_percentage,
            customer_type_cd,
            bbf_ban
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, 0, %s, %s, %s
        ) RETURNING *
    """,
        (
            sf_account_info["name"],
            customer_id,
            company_cd,
            sf_account_info.get("billingstreet"),
            sf_account_info.get("billingcity"),
            sf_account_info.get("billingstate"),
            zip_cd,
            zip4,
            sf_account_info.get("email_address__c"),
            0.015,  # Default late fee percentage
            customer_type_cd,
            True,  # bbf_ban is always True for BBF accounts
        ),
    )

    return ocur.fetchone(), None


def create_sf_billing_account(
    oss_account_info, customer_sfid, legacy_billing_sfid, cur
):
    """
    Creates a new SF Billing Invoice record in Heroku.
    """
    # Build account name with -BBF suffix
    new_ban_name = f"A{oss_account_info['account_id']}-BBF"

    # Determine invoice delivery preference
    paperless = oss_account_info.get("paperless", False)
    email_non_paperless = oss_account_info.get("email_non_paperless", False)

    if paperless:
        delivery_pref = "E-mail"
    elif not paperless and email_non_paperless:
        delivery_pref = "E-mail & Paper"
    else:
        delivery_pref = "Paper"

    cur.execute(
        """
        INSERT INTO sfprod.billing_invoice__c (
            billing_city__c,
            billing_address_1__c,
            account__c,
            account_number__c,
            name,
            late_fee_percentage__c,
            billing_zip__c,
            billing_state__c,
            account_name__c,
            billing_address_2__c,
            disable_late_fees__c,
            suppress_invoice_generation__c,
            payment_terms__c,
            suppress_past_due_notifications__c,
            invoice_delivery_preference__c,
            billing_e_mail__c,
            bbf_ban__c,
            legacy_es_id__c
        ) VALUES (
            %(city)s,
            %(address1)s,
            %(customer_sfid)s,
            %(account_id)s,
            %(ban_name)s,
            %(late_fee_percentage)s,
            %(zip)s,
            %(state_cd)s,
            %(account_nm)s,
            %(address2)s,
            %(disable_late_fees)s,
            false,
            %(due_date_frequency_cd)s,
            %(no_past_due_notice)s,
            %(delivery_pref)s,
            %(billing_email)s,
            true,
            %(legacy_billing_sfid)s
        )
        ON CONFLICT (account_number__c) DO NOTHING
    """,
        {
            "city": oss_account_info.get("city"),
            "address1": oss_account_info.get("address1"),
            "customer_sfid": customer_sfid,
            "account_id": str(oss_account_info["account_id"]),
            "ban_name": new_ban_name,
            "late_fee_percentage": oss_account_info.get("late_fee_percentage", 0.015),
            "zip": oss_account_info.get("zip"),
            "state_cd": oss_account_info.get("state_cd"),
            "account_nm": oss_account_info.get("account_nm"),
            "address2": oss_account_info.get("address2"),
            "disable_late_fees": oss_account_info.get("disable_late_fees", False),
            "due_date_frequency_cd": oss_account_info.get(
                "due_date_frequency_cd", "NET30"
            ),
            "no_past_due_notice": oss_account_info.get("no_past_due_notice", False),
            "delivery_pref": delivery_pref,
            "billing_email": oss_account_info.get("billing_email"),
            "legacy_billing_sfid": legacy_billing_sfid,
        },
    )


def poll_for_sfid(account_id, customer_sfid, cur, max_retries=5, sleep_seconds=5):
    """
    Polls Heroku waiting for Salesforce to assign an SFID to the new billing invoice.
    """
    new_billing_acct = None
    retry_count = 0

    while retry_count <= max_retries and (
        new_billing_acct is None or new_billing_acct.get("sfid") is None
    ):
        retry_count += 1
        cur.execute(
            """
            SELECT sfid, account__c, account_number__c, name
            FROM sfprod.billing_invoice__c
            WHERE account__c = %s
              AND account_number__c = %s
        """,
            (customer_sfid, str(account_id)),
        )

        billing_acct = cur.fetchone()
        if billing_acct and billing_acct.get("sfid"):
            new_billing_acct = billing_acct
        else:
            time.sleep(sleep_seconds)

    return new_billing_acct

In [26]:
# === STEP 9: Process each billing account ===

results = []
skipped_wd_only = []
skipped_already_has_bbf = []

# Record skipped WD-only BANs
for ba in wd_only:
    skipped_wd_only.append(
        {
            "old_account_id": ba["old_account_id"],
            "old_billing_invoice_sfid": ba["old_billing_invoice_sfid"],
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "wd_order_count": ba["wd_order_count"],
            "bbf_order_count": ba["bbf_order_count"],
            "reason": "WD-only: All orders are PA MARKET DECOM",
        }
    )

# Record skipped already-has-BBF BANs
for ba in already_has_bbf:
    existing = existing_bbf_df[
        existing_bbf_df["old_ban_sfid"] == ba["old_billing_invoice_sfid"]
    ].iloc[0]
    skipped_already_has_bbf.append(
        {
            "old_account_id": ba["old_account_id"],
            "old_billing_invoice_sfid": ba["old_billing_invoice_sfid"],
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "existing_bbf_name": existing["existing_bbf_name"],
            "existing_bbf_sfid": existing["existing_bbf_sfid"],
            "reason": "Already has -BBF BAN",
        }
    )

print(f"Recorded {len(skipped_wd_only)} WD-only BANs as skipped")
print(f"Recorded {len(skipped_already_has_bbf)} already-has-BBF BANs as skipped")

# Process BANs that need BBF accounts
total = len(bans_needing_bbf_account)

for idx, ba in enumerate(bans_needing_bbf_account, 1):
    old_account_id = ba["old_account_id"]
    customer_sfid = ba["customer_sfid"]
    old_billing_sfid = ba["old_billing_invoice_sfid"]
    wd_count = ba["wd_order_count"]
    bbf_count = ba["bbf_order_count"]

    # Determine market type
    if bbf_count > 0 and wd_count == 0:
        market_type = "BBF_ONLY"
    else:
        market_type = "MIXED"

    # Get SF Account
    sf_account = sf_accounts.get(customer_sfid)
    if not sf_account:
        result = {
            "old_account_id": old_account_id,
            "old_billing_invoice_sfid": old_billing_sfid,
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "market_type": market_type,
            "wd_order_count": wd_count,
            "bbf_order_count": bbf_count,
            "new_account_id": None,
            "new_ban_name": None,
            "new_billing_invoice_sfid": None,
            "status": "SKIPPED",
            "error": "SF Account not found",
        }
        results.append(result)
        continue

    # Get OSS Customer
    oss_customer = oss_customers.get(customer_sfid)
    if not oss_customer:
        result = {
            "old_account_id": old_account_id,
            "old_billing_invoice_sfid": old_billing_sfid,
            "account_name": ba["account_name__c"],
            "old_ban_name": ba["old_ban_name"],
            "market_type": market_type,
            "wd_order_count": wd_count,
            "bbf_order_count": bbf_count,
            "new_account_id": None,
            "new_ban_name": None,
            "new_billing_invoice_sfid": None,
            "status": "SKIPPED",
            "error": "OSS Customer not found",
        }
        results.append(result)
        continue

    customer_id = oss_customer["customer_id"]

    result = {
        "old_account_id": old_account_id,
        "old_billing_invoice_sfid": old_billing_sfid,
        "account_name": ba["account_name__c"],
        "old_ban_name": ba["old_ban_name"],
        "market_type": market_type,
        "wd_order_count": wd_count,
        "bbf_order_count": bbf_count,
        "new_account_id": None,
        "new_ban_name": None,
        "new_billing_invoice_sfid": None,
        "status": None,
        "error": None,
    }

    # === DRY RUN ===
    if DRY_RUN:
        entity = sf_account.get("onecommunity_entity__c")
        if entity not in entity_map:
            result["status"] = "WOULD_SKIP"
            result["error"] = f"Unknown entity: {entity}"
        else:
            result["new_account_id"] = "(pending)"
            result["new_ban_name"] = "A(pending)-BBF"
            result["new_billing_invoice_sfid"] = "(pending)"
            result["status"] = "WOULD_CREATE"

        results.append(result)
        continue

    # === LIVE RUN ===
    try:
        print(
            f"[{idx}/{total}] Creating -BBF account for {ba['account_name__c']} (old: {old_account_id})"
        )

        # Step 1: Create OSS Account
        with oconn.cursor(cursor_factory=RealDictCursor) as ocur:
            new_oss_account, error = create_oss_account(sf_account, customer_id, ocur)

            if error:
                result["status"] = "SKIPPED"
                result["error"] = error
                results.append(result)
                oconn.rollback()
                continue

            new_account_id = new_oss_account["account_id"]
            result["new_account_id"] = new_account_id
            result["new_ban_name"] = f"A{new_account_id}-BBF"

            oconn.commit()
            print(f"  ✅ Created OSS account: {new_account_id} (bbf_ban=True)")

        # Step 2: Create SF Billing Invoice in Heroku
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            create_sf_billing_account(
                new_oss_account, customer_sfid, old_billing_sfid, cur
            )
            conn.commit()
            print(f"  ✅ Created SF Billing Invoice: A{new_account_id}-BBF")

        # Step 3: Poll for SFID
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            new_billing_acct = poll_for_sfid(new_account_id, customer_sfid, cur)

            if new_billing_acct and new_billing_acct.get("sfid"):
                result["new_billing_invoice_sfid"] = new_billing_acct["sfid"]
                result["status"] = "CREATED"
                print(f"  ✅ Got SFID: {new_billing_acct['sfid']}")
            else:
                result["status"] = "CREATED_NO_SFID"
                result["error"] = "Timed out waiting for SFID"
                print(f"  ⚠️ Created but no SFID yet")

    except Exception as e:
        result["status"] = "ERROR"
        result["error"] = str(e)
        oconn.rollback()
        conn.rollback()
        print(f"  ❌ Error: {e}")

    results.append(result)

    if idx % 100 == 0:
        print(f"Processed {idx}/{total} billing accounts...")

# Summary
print("\n" + "=" * 50)
print("SUMMARY")
print("=" * 50)
status_counts = defaultdict(int)
for r in results:
    status_counts[r["status"]] += 1

for status, count in sorted(status_counts.items()):
    print(f"{status}: {count}")

bbf_acct_count = len(
    [
        r
        for r in results
        if r["status"] in ("WOULD_CREATE", "CREATED", "CREATED_NO_SFID")
    ]
)
print(f"\n-BBF accounts to create: {bbf_acct_count}")
print(f"WD-only BANs skipped: {len(skipped_wd_only)}")
print(f"Already-has-BBF BANs skipped: {len(skipped_already_has_bbf)}")

Recorded 17 WD-only BANs as skipped
Recorded 0 already-has-BBF BANs as skipped
[1/2501] Creating -BBF account for AIS Realtime Locations Solutions Inc (old: 100088)
  ✅ Created OSS account: 117300 (bbf_ban=True)
  ✅ Created SF Billing Invoice: A117300-BBF
  ✅ Got SFID: aA3Qp000000A6XBKA0
[2/2501] Creating -BBF account for Peckham (old: 100097)
  ✅ Created OSS account: 117301 (bbf_ban=True)
  ✅ Created SF Billing Invoice: A117301-BBF
  ✅ Got SFID: aA3Qp000000A6YnKAK
[3/2501] Creating -BBF account for Ingham County (old: 100099)
  ✅ Created OSS account: 117302 (bbf_ban=True)
  ✅ Created SF Billing Invoice: A117302-BBF
  ✅ Got SFID: aA3Qp000000A6aPKAS
[4/2501] Creating -BBF account for Buckeye Telesystem (old: 100101)
  ✅ Created OSS account: 117303 (bbf_ban=True)
  ✅ Created SF Billing Invoice: A117303-BBF
  ✅ Got SFID: aA3Qp000000A6c1KAC
[5/2501] Creating -BBF account for LakeNet (old: 100152)
  ✅ Created OSS account: 117304 (bbf_ban=True)
  ✅ Created SF Billing Invoice: A117304-BBF
  ✅

InterfaceError: connection already closed

In [27]:
# === STEP 10: Create output Excel file ===

wb = Workbook()

# Styling
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
header_fill_green = PatternFill("solid", fgColor="70AD47")
header_fill_orange = PatternFill("solid", fgColor="ED7D31")
header_fill_red = PatternFill("solid", fgColor="C00000")
header_alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)


def style_header(ws, headers, fill_color):
    for col, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col)
        cell.font = header_font
        cell.fill = fill_color
        cell.alignment = header_alignment
        cell.border = thin_border


def auto_width(ws):
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        ws.column_dimensions[column].width = min(max_length + 2, 40)


# Status colors
status_colors = {
    "WOULD_CREATE": "C6EFCE",
    "WOULD_SKIP": "FCE4D6",
    "CREATED": "C6EFCE",
    "CREATED_NO_SFID": "FFEB9C",
    "SKIPPED": "FFCCCC",
    "ERROR": "FF6666",
}

# === Sheet 1: Summary ===
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["ES to BBF BAN Creation Summary (v9)"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE RUN"])
ws1.append(
    ["Fix Missing Customer Links:", "Yes" if FIX_MISSING_CUSTOMER_LINKS else "No"]
)
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append([])
ws1.append(["v9 Features:"])
ws1["A7"].font = Font(bold=True)
ws1.append(["  - Idempotency: Skips BANs that already have -BBF counterpart"])
ws1.append(["  - Customer Link Fix: Can fix missing salesforce_id in OSS"])
ws1.append([])
ws1.append(["BAN Breakdown:"])
ws1["A11"].font = Font(bold=True)
ws1.append(["  Already has -BBF (skipped):", len(skipped_already_has_bbf)])
ws1.append(["  WD-only (skipped):", len(skipped_wd_only)])
ws1.append(["  BBF-only:", len(bbf_only)])
ws1.append(["  Mixed:", len(mixed)])
ws1.append([])
ws1.append(["Accounts to Create:"])
ws1["A17"].font = Font(bold=True)
ws1.append(["  -BBF accounts:", bbf_acct_count])
ws1.append([])
ws1.append(["Customer Link Fixes:"])
ws1["A20"].font = Font(bold=True)
ws1.append(["  Fixes needed:", len(customer_link_fixes)])
ws1.append([])
ws1.append(["Status Counts:"])
ws1["A23"].font = Font(bold=True)
for status, count in sorted(status_counts.items()):
    ws1.append([f"  {status}:", count])

# === Sheet 2: BBF Clone Results ===
ws2 = wb.create_sheet("BBF Clone Results")
headers2 = [
    "Old Account ID",
    "New Account ID",
    "Account Name",
    "Old BAN Name",
    "New BAN Name",
    "Market Type",
    "WD Orders (Stay)",
    "BBF Orders (Move)",
    "Old Billing SFID",
    "New Billing SFID",
    "Status",
    "Error",
]
ws2.append(headers2)
style_header(ws2, headers2, header_fill)

for row_idx, r in enumerate(results, 2):
    ws2.append(
        [
            r["old_account_id"],
            r["new_account_id"],
            r["account_name"],
            r["old_ban_name"],
            r["new_ban_name"],
            r["market_type"],
            r["wd_order_count"],
            r["bbf_order_count"],
            r["old_billing_invoice_sfid"],
            r["new_billing_invoice_sfid"],
            r["status"],
            r["error"],
        ]
    )
    fill_color = status_colors.get(r["status"], "FFFFFF")
    for col in range(1, len(headers2) + 1):
        ws2.cell(row=row_idx, column=col).fill = PatternFill(
            "solid", fgColor=fill_color
        )

ws2.freeze_panes = "A2"
auto_width(ws2)

# === Sheet 3: Skipped - Already Has BBF ===
ws3 = wb.create_sheet("Skipped - Already Has BBF")
headers3 = [
    "Old Account ID",
    "Account Name",
    "Old BAN Name",
    "Existing BBF Name",
    "Existing BBF SFID",
    "Reason",
]
ws3.append(headers3)
style_header(ws3, headers3, header_fill_green)

for r in skipped_already_has_bbf:
    ws3.append(
        [
            r["old_account_id"],
            r["account_name"],
            r["old_ban_name"],
            r["existing_bbf_name"],
            r["existing_bbf_sfid"],
            r["reason"],
        ]
    )

ws3.freeze_panes = "A2"
auto_width(ws3)

# === Sheet 4: Skipped - WD Only ===
ws4 = wb.create_sheet("Skipped - WD Only")
headers4 = [
    "Old Account ID",
    "Account Name",
    "Old BAN Name",
    "WD Orders",
    "BBF Orders",
    "Old Billing SFID",
    "Reason",
]
ws4.append(headers4)
style_header(ws4, headers4, header_fill_orange)

for r in skipped_wd_only:
    ws4.append(
        [
            r["old_account_id"],
            r["account_name"],
            r["old_ban_name"],
            r["wd_order_count"],
            r["bbf_order_count"],
            r["old_billing_invoice_sfid"],
            r["reason"],
        ]
    )

ws4.freeze_panes = "A2"
auto_width(ws4)

# === Sheet 5: Customer Link Fixes ===
ws5 = wb.create_sheet("Customer Link Fixes")
headers5 = [
    "Customer ID",
    "Customer Name",
    "Old Account ID",
    "SF Account SFID",
    "Current Salesforce ID",
    "Status",
]
ws5.append(headers5)
style_header(ws5, headers5, header_fill)

for r in customer_link_fixes:
    ws5.append(
        [
            r["customer_id"],
            r["customer_nm"],
            r["old_account_id"],
            r["sf_account_sfid"],
            r["current_salesforce_id"],
            r["status"],
        ]
    )

ws5.freeze_panes = "A2"
auto_width(ws5)

# === Sheet 6: Errors ===
ws6 = wb.create_sheet("Errors")
headers6 = ["Old Account ID", "Account Name", "Market Type", "Status", "Error"]
ws6.append(headers6)
style_header(ws6, headers6, header_fill_red)

for r in results:
    if r["status"] in ("SKIPPED", "WOULD_SKIP", "ERROR"):
        ws6.append(
            [
                r["old_account_id"],
                r["account_name"],
                r["market_type"],
                r["status"],
                r["error"],
            ]
        )

ws6.freeze_panes = "A2"
auto_width(ws6)

# Save
wb.save(output_file)
print(f"\n✅ Output saved to: {output_file}")


✅ Output saved to: ./es_to_bbf_ban_creation_v9_live_run_20260106_093058.xlsx


In [28]:
# === Cleanup ===
conn.close()
oconn.close()
print("🔌 Database connections closed.")

if DRY_RUN:
    print("\n🔄 DRY RUN complete. No accounts were created.")
    print(f"Review the output file: {output_file}")
    print("\nTo run for real, set DRY_RUN = False and re-run the notebook.")
    if customer_link_fixes:
        print(f"\n⚠️ {len(customer_link_fixes)} customer link fixes needed.")
        print("   Set FIX_MISSING_CUSTOMER_LINKS = True to apply fixes.")
else:
    print(f"\n✅ LIVE RUN complete.")
    print(f"Created {bbf_acct_count} -BBF accounts.")
    print(f"Skipped {len(skipped_wd_only)} WD-only BANs.")
    print(f"Skipped {len(skipped_already_has_bbf)} already-has-BBF BANs.")
    print(f"Output saved to: {output_file}")

🔌 Database connections closed.

✅ LIVE RUN complete.
Created 2501 -BBF accounts.
Skipped 17 WD-only BANs.
Skipped 0 already-has-BBF BANs.
Output saved to: ./es_to_bbf_ban_creation_v9_live_run_20260106_093058.xlsx
